# 7. GUI creation

pyqt and pyqtgraph & pymmcore-widgets

## pyqt and pyqtgraph <a id='pyqt_and_pyqtgraph'></a>

In [4]:
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QWidget, QGridLayout
from PyQt5.QtGui import QFont
import pyqtgraph as pg
import numpy as np

In [5]:
### simple plot

# start application
app = pg.mkQApp()

# create window
plot_widget = pg.PlotWidget()
x = np.linspace(0, 20, 1000)
y = np.sin(x)
plot_widget.plot(x, y)
plot_widget.show()

# execute application
app.exec()


0

In [6]:
### moving sine curve ('live data acquisition')

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.t = np.linspace(0, 2*np.pi, 1000)
        self.num = 0.01
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph, 0, 0, 1, 1)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.setGeometry(0, 0, 500, 200)
        self.show()

    def update(self):
        self.graph.plotItem.clear()
        data = np.sin(self.t + self.num)
        self.graph.plotItem.plot(self.t, data)
        self.num = self.num + 0.5

app = QApplication(['auto update'])

win = window()

app.exec_()

0

In [ ]:
### display live microscopy images

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.show()
    
    def update(self):
        core.setConfig('LED_light', 'on')
        self.graph.plotItem.clear()
        core.snapImage()
        im = core.getImage()
        imv = pg.ImageView()
        imv.setImage(im)
        imv.show()
        #imv.close()
        core.setConfig('LED_light', 'off')

app = QApplication(['auto update'])

win = window()

app.exec_()

In [ ]:
### PyQt5 tutorial

import PyQt5.QtWidgets as qtw
import PyQt5.QtGui as qtg

class MainWindow(qtw.QWidget):
    def __init__(self):
        super().__init__()
        
        # title
        self.setWindowTitle('Microscope Automation')
        
        # layout
        self.setLayout(qtw.QHBoxLayout())
        
        # label
        label = qtw.QLabel('What`s your name?')
        # change font size of label
        label.setFont(qtg.QFont('SansSerif', 30))
        self.layout().addWidget(label)
        
        # create entry box
        entry = qtw.QLineEdit()
        entry.setObjectName('name_field')
        entry.setText('')
        self.layout().addWidget(entry)
        # create a button
        button = qtw.QPushButton('Press me!')
        
        
        self.show()

app = qtw.QApplication([])
mw = MainWindow()

app.exec_()

### pymmcore-widgets

In [ ]:
### some widgets

from qtpy.QtWidgets import QApplication
from pymmcore_widgets import PropertyBrowser, LiveButton, ExposureWidget, ImagePreview
app = QApplication([])

# create a PropertyBrowser widget. By default, this widget will use the active
# Micro-Manager core instance.

pb_widget = PropertyBrowser()
pb_widget.show()

live_btn = LiveButton()
live_btn.show()

exp_wdg = ExposureWidget()
exp_wdg.show()

img_wdg = ImagePreview()
img_wdg.show()

app.exec_()

In [ ]:
### preview widget with buttons

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus

from qtpy.QtWidgets import QApplication, QGroupBox, QHBoxLayout, QVBoxLayout, QWidget

from pymmcore_widgets import (
    ChannelWidget,
    ExposureWidget,
    ImagePreview,
    LiveButton,
    SnapButton,
    ChannelGroupWidget,
)

class ImageFrame(QWidget):
    """An example widget with a snap/live button and an image preview."""

    def __init__(self):
        super().__init__()

        self.preview = ImagePreview()
        self.snap_button = SnapButton()
        self.live_button = LiveButton()
        self.exposure = ExposureWidget()
        self.channel = ChannelWidget()
        self.channel_group = ChannelGroupWidget()

        self.setLayout(QVBoxLayout())

        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(self.snap_button)
        buttons.layout().addWidget(self.live_button)

        ch_exp = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        ch_exp.setLayout(layout)

        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().setContentsMargins(0, 0, 0, 0)
        ch.layout().addWidget(self.channel)
        layout.addWidget(ch)

        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().setContentsMargins(0, 0, 0, 0)
        ch_gr.layout().addWidget(self.channel_group)
        layout.addWidget(ch_gr)

        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().setContentsMargins(0, 0, 0, 0)
        exp.layout().addWidget(self.exposure)
        layout.addWidget(exp)

        self.layout().addWidget(self.preview)
        self.layout().addWidget(ch_exp)
        self.layout().addWidget(buttons)


if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    #core.loadSystemConfiguration()
    app = QApplication([])
    frame = ImageFrame()
    frame.show()
    #core.snap()
    app.exec_()

![Preview Window](preview_window.png "Title")

##### MainWindow with multiple widgets

In [1]:
### functions ###

from scipy.optimize import curve_fit
import m2stitch

def gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def _software_autofocus(core, range, step_size):
    z = core.getZPosition()

    # define location and type of image saving
    writer = ImageSequenceWriter(r'C:\Users\Admin\Desktop\focus', extension=".png", overwrite=True)

    pos = core.getXYPosition()
    # acquire z-stack
    sequence = MDASequence(
        axis_order="tpgcz",
        stage_positions=[(pos[0], pos[1], z)],
        channels=[{'group': 'LED_light', 'config': 'on'}],
        z_plan = {'above': range, 'below': range, 'step': step_size}
    )

    # run focus mda sequence
    with mda_listeners_connected(writer):
        core.mda.run(sequence)

    # calculate focus scores
    focus_images = glob.glob(r'C:\Users\Admin\Desktop\focus\*.png')
    focus_scores = []
    for f in focus_images:
        im = cv2.imread(rf'{f}')
        im_filtered = cv2.medianBlur(im, ksize=3)
        laplacian = cv2.Laplacian(im_filtered, ddepth=cv2.CV_64F, ksize=3)
        focus_score = laplacian.var()
        focus_scores.append(focus_score)

    # define x and y for fitting
    y = focus_scores
    x = np.linspace(z-range, z+range, len(y))

    # define gauss fit function
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))

    # optimize gauss fit
    popt, pcov = curve_fit(gauss, x, y, p0 = [np.max(y), mean, sigma])

    # calculate maximum focus score of fit function
    x_fine = np.linspace(z-range, z+range, 100)
    y_max_fit = np.max(gauss(x_fine,*popt))
    pos = np.where(y_max_fit == gauss(x_fine,*popt))
    x_max_fit = x_fine[pos]
    y_max_data = np.max(focus_scores)
    x_max_data = x[np.where(focus_scores == y_max_data)]

    # set optimal z position
    core.setPosition(float(x_max_fit[0]))

    # remove focus images
    for file in focus_images:
        os.remove(file)

def _run_acquisition_sequence(core, rows, columns, overlap, fov_width=1192.8):
    z = core.getZPosition()
    sequence = MDASequence(axis_order = "tpgcz",
                           stage_positions = [(1000, 1500, z)],
                           grid_plan = {"rows": rows, "columns": columns, 
                                        'relative_to': 'center',
                                        'overlap': overlap,
                                        'mode': 'row_wise',
                                        'fov_width': fov_width,
                                        'fov_height': fov_width},
                           channels = [{'group': 'LED_light', 'config': 'on'}])
    png_writer = ImageSequenceWriter(r'C:\Users\Admin\Desktop\test_files', 
                                     extension=".png", 
                                     overwrite=True)
    with mda_listeners_connected(png_writer):
        core.mda.run(sequence)
    
def _stitching(folder=r'C:\Users\Admin\Desktop\test_files\*.png'):
    image_files = glob.glob(rf'{folder}')
    images = []
    for i in image_files:
        images.append(cv2.imread(i[0]))
    images = np.stack(images, axis=0)
    
    def flatten_rows(rows):
        r = []
        for xs in rows:
            for x in xs:
                r.append(x)
        return r

    def flatten_cols(cols):
        c = np.concatenate(cols).ravel().tolist()
        return c
    
    grid_size = 2
    rows = []
    cols = []
    for r in range(grid_size):
        rows.append([r+1]*(grid_size))
        cols.append([np.arange(1, grid_size+1)])
    
    c = flatten_cols(cols)
    r = flatten_rows(rows)
    
    result_df, _ = m2stitch.stitch_images(
        images, r, c, 
        row_col_transpose=False,
        ncc_threshold=0.05)
    
    result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
    result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()
    size_y, size_x = images.shape[1], images.shape[2]
    stitched_image_size = (
        result_df["y_pos2"].max() + size_y,
        result_df["x_pos2"].max() + size_x,
    )
    stitched_image = np.zeros_like(images, shape=stitched_image_size)
    for i, row in result_df.iterrows():
        stitched_image[
            row["y_pos2"] : row["y_pos2"] + size_y,
            row["x_pos2"] : row["x_pos2"] + size_x,
        ] = images[i]
    
    ### display ###
    #plt.imshow(stitched_image, cmap='grey')

### multiple widgets in main window ###

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus
import PyQt5.QtWidgets as qtw
from qtpy.QtWidgets import QStackedWidget, QMainWindow, QAction, QApplication, QGroupBox, QWidget, QHBoxLayout, QVBoxLayout, QGridLayout
import pymmcore_widgets as pycw
from useq import MDAEvent, MDASequence, Position
from pymmcore_plus.mda import mda_listeners_connected
from pymmcore_plus.mda.handlers import ImageSequenceWriter
import glob
import cv2
import numpy as np

          
class MainWindow(qtw.QWidget):
    
    def __init__(self):
        
        super().__init__()
        
        self.setLayout(qtw.QHBoxLayout())
        self.setWindowTitle('Microscope Automation')
        self.setGeometry(0, 0, 1500, 1000)
        #self.setStyleSheet("background-color: darkgrey;") 
        
        ### 1) preview ###
        preview = QGroupBox()
        preview.setLayout(QGridLayout())
        preview.setTitle('Preview')
        
        # define widgets #
        stage_XY = pycw.StageWidget('XYStage')
        stage_Z = pycw.StageWidget('ZStage')
        snap_button = pycw.SnapButton()
        live_button = pycw.LiveButton()
        image_preview = pycw.ImagePreview()
        exposure = pycw.ExposureWidget()
        channel_group = pycw.ChannelGroupWidget()
        channel = pycw.ChannelWidget()
        
        # stage #
        stage = QGroupBox()
        stage.setLayout(qtw.QHBoxLayout())
        stage.layout().addWidget(stage_XY)
        stage.layout().addWidget(stage_Z)
        
        # snap & live buttons window #
        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(snap_button)
        buttons.layout().addWidget(live_button)
        
        # channelgroup, channel & exposure window #
        ch_exp = QWidget()
        layout = QHBoxLayout()
        ch_exp.setLayout(layout)
        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().addWidget(channel_group)
        layout.addWidget(ch_gr)
        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().addWidget(channel)
        layout.addWidget(ch)
        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().addWidget(exposure)
        layout.addWidget(exp)
        
        # combine preview window
        preview.layout().addWidget(image_preview, 0, 0)
        preview.layout().addWidget(ch_exp, 1, 0)
        preview.layout().addWidget(buttons, 2, 0)
        preview.layout().addWidget(stage, 3, 0)
        
        ### 2) middle ###
        middle = QWidget()
        layout = QVBoxLayout()
        middle.setLayout(layout)
        
        ## autofocus ##
        autofocus = QGroupBox()
        autofocus.setLayout(QGridLayout())
        autofocus.setTitle('Autofocus')
        # range #
        af_range = qtw.QDoubleSpinBox(value = 10, maximum = 100, minimum = 5, singleStep = 5)
        # step size #
        af_step_size = qtw.QDoubleSpinBox(value = 2, maximum = 50, singleStep = 1)
        # autofocus button #
        autofocus_button = qtw.QPushButton('Autofocus', 
                                           clicked = lambda: software_autofocus(self.core))
        # combine autofocus #
        autofocus.layout().addWidget(qtw.QLabel('range [µm]'), 0, 0)
        autofocus.layout().addWidget(af_range, 0, 1)
        autofocus.layout().addWidget(qtw.QLabel('step size [µm]'), 1, 0)
        autofocus.layout().addWidget(af_step_size, 1, 1)
        autofocus.layout().addWidget(autofocus_button, 2, 1)
        
        ## tile scan ##
        tile_scan = QGroupBox()
        tile_scan.setLayout(QGridLayout())
        tile_scan.setTitle('Tile scan')
        # grid size #
        grid_size = qtw.QDoubleSpinBox(value = 1, maximum = 100, minimum = 1, singleStep = 1)
        # overlap #
        overlap = qtw.QDoubleSpinBox(value = 10, maximum = 100, minimum = 1, singleStep = 5)
        # stitching checkbox #
        stitching_checkbox = qtw.QCheckBox()
        stitching_checkbox.setChecked(True)
        #stitching_checkbox.stateChanged.connect(stitching())
        # run sequence button #
        #run_sequence_button = qtw.QPushButton('run acquisition', 
        #                                      clicked = lambda: run_acquisition_sequence(self.core))
        # browse field for save folder #
        
        # combine tile scan #
        tile_scan.layout().addWidget(qtw.QLabel('grid size'), 0, 0)
        tile_scan.layout().addWidget(grid_size, 0, 1)
        tile_scan.layout().addWidget(qtw.QLabel('overlap [%]'), 1, 0)
        tile_scan.layout().addWidget(overlap, 1, 1)
        tile_scan.layout().addWidget(qtw.QLabel('stitching and display?'), 2, 0)
        tile_scan.layout().addWidget(stitching_checkbox, 2, 1)
        
        ## Multi-well tool ##
        multi_well = QGroupBox()
        multi_well.setTitle('Multi-well tool')
        multi_well.setLayout(QHBoxLayout())
        layout.addWidget(multi_well)
        
        ## overview scan ##
        overview_scan = QGroupBox()
        overview_scan.setTitle('Overview scan')
        overview_scan.setLayout(QHBoxLayout())
        layout.addWidget(overview_scan)
        
        # combine autofocus window #
        middle.layout().addWidget(autofocus)
        middle.layout().addWidget(tile_scan)
        middle.layout().addWidget(multi_well)
        middle.layout().addWidget(overview_scan)
        
        ### 3) bath perfusion ###
        bath_perfusion = QWidget()
        layout = QVBoxLayout()
        bath_perfusion.setLayout(layout)
        bath_perfusion_tool = QGroupBox()
        bath_perfusion_tool.setTitle('Bath perfusion tool')
        bath_perfusion_tool.setLayout(QHBoxLayout())
        layout.addWidget(bath_perfusion_tool)
        
        self.core = None
        self.layout().addWidget(preview)
        self.layout().addWidget(middle)
        self.layout().addWidget(bath_perfusion)
        
        self.show()
    
        def software_autofocus(core, range = af_range.value(), step_size = af_step_size.value()):
            _software_autofocus(core = core, range = range, step_size = step_size)
            
        #def run_acquisition_sequence(rows=grid_size.value(), 
        #                             columns=grid_size.value(), 
        #                             overlap=overlap.value(), 
        #                             fov_width=1192.8):
        #    _run_acquisition_sequence(core, rows, columns, overlap, fov_width=1192.8)
        #
        #def stitching():
        #    _stitching()

if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    app = qtw.QApplication([])
    mw = MainWindow()
    mw.core = core
    app.exec_()

2024-08-29 20:02:11,053 - pymmcore-plus - INFO - (_runner.py:321) MDA Started: stage_positions=(Position(x=1200.0, y=1214.0, z=-906.0, name=None, sequence=None),) channels=(Channel(config='on', group='LED_light', exposure=None, do_stack=True, z_offset=0.0, acquire_every=1, camera=None),) z_plan=ZAboveBelow(go_up=True, above=10.0, below=10.0, step=2.0)
2024-08-29 20:02:11,057 - pymmcore-plus - INFO - (_runner.py:283) index=mappingproxy({'p': 0, 'c': 0, 'z': 0}) channel=Channel(config='on', group='LED_light') x_pos=1200.0 y_pos=1214.0 z_pos=-916.0
2024-08-29 20:02:11,708 - pymmcore-plus - INFO - (_runner.py:283) index=mappingproxy({'p': 0, 'c': 0, 'z': 1}) channel=Channel(config='on', group='LED_light') x_pos=1200.0 y_pos=1214.0 z_pos=-914.0
2024-08-29 20:02:12,374 - pymmcore-plus - INFO - (_runner.py:283) index=mappingproxy({'p': 0, 'c': 0, 'z': 2}) channel=Channel(config='on', group='LED_light') x_pos=1200.0 y_pos=1214.0 z_pos=-912.0
2024-08-29 20:02:13,058 - pymmcore-plus - INFO - (_r

![window](window.png)

In [4]:
mw.af_range.value()

AttributeError: 'MainWindow' object has no attribute 'af_range'

In [1]:
### turn LED light off

from pymmcore_plus import CMMCorePlus
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
import os
core = CMMCorePlus()
core.setDeviceAdapterSearchPaths([mm_dir])
core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
core.setConfig('LED_light', 'off')

In [1]:
### functions


def gauss(x, a, x0, sigma):
        return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def _software_autofocus(core, range, step_size):

        #xx, yy = core.getXYPosition()
        z = core.getZPosition()
    
        # define location and type of image saving
        writer = ImageSequenceWriter(r'C:\Users\Admin\Desktop\focus', extension=".png", overwrite=True)
    
        # acquire z-stack
        sequence = MDASequence(
            axis_order="tpgcz",
            stage_positions=[(900, 1214, z)],
            channels=[{'group': 'LED_light', 'config': 'on'}],
            z_plan={'above': range, 'below': range, 'step': step_size}
        )
    
        # run focus mda sequence
        with mda_listeners_connected(writer):
            core.mda.run(sequence)
    
        # calculate focus scores
        focus_images = glob.glob(r'C:\Users\Admin\Desktop\focus\*.png')
        focus_scores = []
        for f in focus_images:
            im = cv2.imread(rf'{f}')
            im_filtered = cv2.medianBlur(im, ksize=3)
            laplacian = cv2.Laplacian(im_filtered, ddepth=cv2.CV_64F, ksize=3)
            focus_score = laplacian.var()
            focus_scores.append(focus_score)
    
        # define x and y for fitting
        y = focus_scores
        x = np.linspace(z-range, z+range, len(y))
    
        # define gauss fit function
        mean = sum(x * y) / sum(y)
        sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))
    
        # optimize gauss fit
        popt, pcov = curve_fit(gauss, x, y, p0 = [np.max(y), mean, sigma])
    
        # calculate maximum focus score of fit function
        x_fine = np.linspace(z-range, z+range, 100)
        y_max_fit = np.max(gauss(x_fine,*popt))
        pos = np.where(y_max_fit == gauss(x_fine,*popt))
        x_max_fit = x_fine[pos]
        y_max_data = np.max(focus_scores)
        x_max_data = x[np.where(focus_scores == y_max_data)]
    
        # set optimal z position
        core.setPosition(float(x_max_fit[0]))
    
        # remove focus images
        for file in focus_images:
            os.remove(file)

### multiple widgets in main window: live view 

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus
import PyQt5.QtWidgets as qtw
from qtpy.QtWidgets import QStackedWidget, QMainWindow, QAction, QApplication, QGroupBox, QWidget, QHBoxLayout, QVBoxLayout
import pymmcore_widgets as pycw
from useq import MDAEvent, MDASequence, Position
from pymmcore_plus.mda import mda_listeners_connected
from pymmcore_plus.mda.handlers import ImageSequenceWriter
import glob
import cv2
import numpy as np
from scipy.optimize import curve_fit
import pyqtgraph as pg
import matplotlib.pyplot as plt

          
class MainWindow(qtw.QWidget):
    
    def __init__(self):
        
        super().__init__()
        
        self.setLayout(qtw.QHBoxLayout())
        self.setWindowTitle('Microscope Automation')
        self.setGeometry(0, 0, 1500, 1000)
        
        ### preview ###
        preview = QGroupBox()
        preview.setLayout(qtw.QVBoxLayout())
        
        # define widgets #
        stage_XY = pycw.StageWidget('XYStage')
        stage_Z = pycw.StageWidget('ZStage')
        snap_button = pycw.SnapButton()
        live_button = pycw.LiveButton()
        image_preview = pycw.ImagePreview()
        exposure = pycw.ExposureWidget()
        channel_group = pycw.ChannelGroupWidget()
        channel = pycw.ChannelWidget()
        
        # stage #
        stage = QGroupBox()
        stage.setLayout(qtw.QHBoxLayout())
        stage.layout().addWidget(stage_XY)
        stage.layout().addWidget(stage_Z)
        
        # snap & live buttons window #
        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(snap_button)
        buttons.layout().addWidget(live_button)
        
        # channelgroup, channel & exposure window #
        ch_exp = QWidget()
        layout = QHBoxLayout()
        ch_exp.setLayout(layout)
        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().addWidget(channel_group)
        layout.addWidget(ch_gr)
        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().addWidget(channel)
        layout.addWidget(ch)
        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().addWidget(exposure)
        layout.addWidget(exp)
        
        # combine preview window
        preview.layout().addWidget(image_preview)
        preview.layout().addWidget(ch_exp)
        preview.layout().addWidget(buttons)
        preview.layout().addWidget(stage)
        
        ### middle ###
        middle = QWidget()
        layout = QVBoxLayout()
        middle.setLayout(layout)
        
        ## software autofocus ##
        software_autofocus = QWidget()
        layout = QHBoxLayout()
        software_autofocus.setLayout(layout)
        # range #
        af_range = QGroupBox()
        af_range.setTitle('range [µm]')
        af_range.setLayout(QHBoxLayout())
        af_range_ = qtw.QDoubleSpinBox(value = 10, maximum = 100, minimum = 5, singleStep = 5)
        af_range.layout().addWidget(af_range_)
        layout.addWidget(af_range)
        # step size #
        af_step_size = QGroupBox()
        af_step_size.setTitle('step size [µm]')
        af_step_size.setLayout(QHBoxLayout())
        af_step_size_ = qtw.QDoubleSpinBox(value = 2, maximum = 10, singleStep = 1)
        af_step_size.layout().addWidget(af_step_size_)
        layout.addWidget(af_step_size)
        # autofocus button #
        autofocus_button = QGroupBox()
        autofocus_button.setTitle('Autofocus')
        autofocus_button.setLayout(QHBoxLayout())
        autofocus_button_ = qtw.QPushButton('Autofocus', 
                                            clicked = lambda: software_autofocus(self.core))
        autofocus_button.layout().addWidget(autofocus_button_)
        layout.addWidget(autofocus_button)
        # combine software autofocus window #
        software_autofocus.layout().addWidget(af_range)
        software_autofocus.layout().addWidget(af_step_size)
        software_autofocus.layout().addWidget(autofocus_button)
        
        ## tile scan ##
        tile_scan = QGroupBox()
        tile_scan.setTitle('Tile scan')
        tile_scan.setLayout(QHBoxLayout())
        layout.addWidget(tile_scan)
        
        ## Multi-well tool ##
        multi_well = QGroupBox()
        multi_well.setTitle('Multi-well tool')
        multi_well.setLayout(QHBoxLayout())
        layout.addWidget(multi_well)
        
        ## overview scan ##
        overview_scan = QGroupBox()
        overview_scan.setTitle('Overview scan')
        overview_scan.setLayout(QHBoxLayout())
        layout.addWidget(overview_scan)
        
        # combine autofocus window #
        middle.layout().addWidget(software_autofocus)
        middle.layout().addWidget(tile_scan)
        middle.layout().addWidget(multi_well)
        middle.layout().addWidget(overview_scan)
        
        ### bath perfusion ###
        bath_perfusion = QWidget()
        layout = QVBoxLayout()
        bath_perfusion.setLayout(layout)
        bath_perfusion_tool = QGroupBox()
        bath_perfusion_tool.setTitle('Bath perfusion tool')
        bath_perfusion_tool.setLayout(QHBoxLayout())
        layout.addWidget(bath_perfusion_tool)
        
        self.core = None
        self.layout().addWidget(preview)
        self.layout().addWidget(middle)
        self.layout().addWidget(bath_perfusion)
        
        self.show()
    
        def software_autofocus(range = af_range_.value(), step_size = af_step_size_.value()):
            _software_autofocus(core, range = range, step_size = step_size)

if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    app = qtw.QApplication([])
    mw = MainWindow()
    mw.core = core
    app.exec_()

ValidationError: 7 validation errors for MDASequence
z_plan.ZTopBottom.top
  Field required [type=missing, input_value={'above': <CMMCorePlus at...9fb0d95e0>, 'step': 2.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
z_plan.ZTopBottom.bottom
  Field required [type=missing, input_value={'above': <CMMCorePlus at...9fb0d95e0>, 'step': 2.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
z_plan.ZAboveBelow.above
  Input should be a valid number [type=float_type, input_value=<CMMCorePlus at 0x2a9fb0d95e0>, input_type=CMMCorePlus]
    For further information visit https://errors.pydantic.dev/2.8/v/float_type
z_plan.ZAboveBelow.below
  Input should be a valid number [type=float_type, input_value=<CMMCorePlus at 0x2a9fb0d95e0>, input_type=CMMCorePlus]
    For further information visit https://errors.pydantic.dev/2.8/v/float_type
z_plan.ZRangeAround.range
  Field required [type=missing, input_value={'